In [20]:
import os
import json
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime

En este jupiter busco unificar los datos relevantes en base a la decisión de elegir trabajar para la cadena hotelera "Holiday Inn". Partiendo desde el dataset de Google [metadata y review-estados]

Tanto las carpetas metadata y review-estados contienen archivos JSON, donde cada linea de estos archivos es un JSON. 
La idea principal es, reducir la cantidad de datos y cambiar el formato de los archivos de JSON a .parquet para poder trabajarlos con menos costo computacional y disponer en primera instancia de los datos que me resultan relevantes de conocer, en este caso de Holiday Inn. 

Para tener una mirada más amplia y prolija sobre la data de cada estado, decidí unificar los archivos 
json dentro de las carpetas de cada estado.

In [2]:
def iterador_linea(archivo_json):
    #archivo_json = "../reviews-estados/review-Alabama/1.json"
    objetos_json = []

    #iteracion del archivo linea por linea, agregando nuevos objetos al array vacio.
    with open(archivo_json, "r") as file:
        for linea in file:
            objeto = json.loads(linea)
            objetos_json.append(objeto)
            
    #creo el dataframe donde se va a alojar esta nueva info.
    df = pd.DataFrame(objetos_json)
    return df

In [3]:
#prueba
df1 = "../reviews-estados/review-Alabama/1.json"
iterador_linea(df1)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,114043824230907811356,Kanisha Mixon,1597168272670,5,Very Personable staff! Beautiful and clean env...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
1,116009008420407071921,Brandie Hodges,1609899039594,5,Best clothing intown,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
2,106239928178034609776,Sharon King,1547235290843,4,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
3,104970056115801744726,Veronica Pierce,1517709403534,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
4,110587471700525107383,Whitney Waldon Collier,1535245718492,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
...,...,...,...,...,...,...,...,...
149995,116508195431461017782,Lee Lee,1601666489107,5,Just a good old hardware friendly people,None,None,0x889a89fd92d55353:0x78e335f5e98e75b2
149996,108020832340640374305,Hayward Salter,1621994613217,5,Good people,None,None,0x889a89fd92d55353:0x78e335f5e98e75b2
149997,105464581418690693091,James McBride,1601863228986,4,Good for locals needing supplies to fix damage...,None,None,0x889a89fd92d55353:0x78e335f5e98e75b2
149998,111886905477779410743,Karen Sullivan,1593013285677,1,Very rude female employees at the cash counter...,None,None,0x889a89fd92d55353:0x78e335f5e98e75b2


Ahora quiero lograr una función que itere archivo por archivo de la carpeta-estado que elija, aplicando la conversión de la función anterior y luego unirlos en un unico dataframe.

In [22]:
def iterador_archivo(carpeta):
    dataframes = []
    #iteracion archivo por archivo, aplicando la función creada anteriormente para transformarlos.
    for archivo in os.listdir(carpeta):
        ruta_archivo = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta_archivo):
            dataframes.append(iterador_linea(ruta_archivo))
    df = pd.concat(dataframes, axis=0, ignore_index=True)
    return df

In [24]:
carpeta = "../reviews-estados/HolidayInn-Review/"
dataframe_final = iterador_archivo(carpeta)
dataframe_final

,user_id,name,time,rating,text,pics,resp,gmap_id,nombre_archivo
0,112665186528292597385,Charlie Cline,2016-02-07 04:58:14.934000+00:00,4,Good location. Nice staff. Clean. Breakfast ar...,None,None,0x88890491e9416625:0x2f85307a06ddb16,review-Alabama.parquet
1,108637957038201408174,Jennifer Passmore,2017-04-12 03:04:16.419000+00:00,5,Super clean very friendly,None,None,0x88890491e9416625:0x2f85307a06ddb16,review-Alabama.parquet
2,109079098695645726629,TheBeckyboo926,2016-10-15 01:14:34.021000+00:00,5,Amazing! We had a conference room and it was a...,None,None,0x88890491e9416625:0x2f85307a06ddb16,review-Alabama.parquet
3,117088716850554427602,yolanda arellano,2016-11-14 14:12:46.153000+00:00,3,A bit on the pricey side but they take on pets...,None,None,0x88890491e9416625:0x2f85307a06ddb16,review-Alabama.parquet
4,117028379331677388352,Greg Randall,2016-05-28 18:39:33.963000+00:00,4,Solid hotel. Staff was friendly. Room was clean.,None,None,0x88890491e9416625:0x2f85307a06ddb16,review-Alabama.parquet
...,...,...,...,...,...,...,...,...,...
1790,105795008554715307308,Mike Coates,2018-01-18 07:36:05.228000+00:00,3,None,None,None,0x5490112537f789dd:0x2e07222ed9e353d8,review-Washington.parquet
1791,110779513713475018373,Sadie Whitener,2018-11-05 16:59:28.344000+00:00,4,None,None,None,0x5490112537f789dd:0x2e07222ed9e353d8,review-Washington.parquet
1792,117261844917157124247,AJ Cox,2016-12-16 12:03:51.446000+00:00,5,None,None,None,0x5490112537f789dd:0x2e07222ed9e353d8,review-Washington.parquet
1793,112699397651929368731,xanvavy,2019-11-14 09:33:36.071000+00:00,5,None,None,None,0x5490112537f789dd:0x2e07222ed9e353d8,review-Washington.parquet


In [5]:
#corroboro que el codigo anterior funcione correctamente:
alabama = "../reviews-estados/review-Alabama"
df = iterador_archivo(alabama)
df

,user_id,name,time,rating,text,pics,resp,gmap_id
0,114043824230907811356,Kanisha Mixon,1597168272670,5,Very Personable staff! Beautiful and clean env...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
1,116009008420407071921,Brandie Hodges,1609899039594,5,Best clothing intown,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
2,106239928178034609776,Sharon King,1547235290843,4,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
3,104970056115801744726,Veronica Pierce,1517709403534,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
4,110587471700525107383,Whitney Waldon Collier,1535245718492,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
...,...,...,...,...,...,...,...,...
1799995,104936410145483423068,Keith Serafin,1599158137870,1,None,None,"{'time': 1599161854585, 'text': 'Hey Keith! P...",0x8889f923aff52f69:0xf42d23231aafe68f
1799996,113333149883363675097,Brenda McClure,1560738061123,5,None,None,None,0x8889f923aff52f69:0xf42d23231aafe68f
1799997,104809929974434632301,Jan Peters,1582415271708,5,None,None,None,0x8889f923aff52f69:0xf42d23231aafe68f
1799998,112113584832789867890,Brittany Banks,1530991256525,5,None,None,None,0x8889f923aff52f69:0xf42d23231aafe68f


Una vez la información del estado unificada en un solo dataframe, me voy a quedar unicamente con todos aquellos registros que daten del 2016 en adelante.
Para esto, tengo que transformar el formato de time, que se encuentra en timestamp Unix (segundos transcurridos desde el 1 de enero de 1970) a formato date YYYY-MM-DD y luego filtrar la informacion con la que me quiero quedar.

In [6]:
def filter_year(dataframe):
    dataframe['time'] = pd.to_datetime(dataframe['time'], unit='ms', utc=True)
    df_filtrado = dataframe[(dataframe['time'].dt.year >= 2016) & (dataframe['time'].dt.year <= 2020)]
    return df_filtrado

Hago una pequeña prueba de que esto corra correctamente.

In [7]:
df_year = filter_year(df)

In [8]:
#se imprimen tanto la menor fecha registrada en el dataframe como la maxima fecha en el mismo y se comparan con el df original.
print("Fecha min original:", df['time'].min())
print("Fecha max original:", df['time'].max())
#--------------------------------------------------Fecha actual:
print("Fecha min Actual:", df_year['time'].min())
print("Fecha max Actual:", df_year['time'].max())

Fecha min original: 2008-01-10 06:37:35.622000+00:00
Fecha max original: 2021-09-09 01:02:47.785000+00:00
Fecha min Actual: 2016-01-01 00:08:34.413000+00:00
Fecha max Actual: 2020-12-31 23:59:30.200000+00:00


Tengo que chequear que, los gmap_id de metadata coincidan con la información de reviews, para esto tengo que traer metadata:

In [9]:
df_met = pd.read_parquet('./metadata')
df_met

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025006,The Green Room Of Libertyville,"The Green Room Of Libertyville, 624 Milwaukee ...",0x880f96c84b0ef57b:0xe75b67b15a2cfdad,"Innovative, globally inspired dishes are paire...",42.289450,-87.954386,"[Restaurant, Bar]",4.6,228,$$,"[[Saturday, 4:30–9PM], [Sunday, Closed], [Mond...","{'Accessibility': None, 'Activities': None, 'A...",Closed ⋅ Opens 4:30PM,"[0x880f96c848329bd5:0x5a6609c274091c81, 0x880f...",https://www.google.com/maps/place//data=!4m2!3...
3025007,Klein Brothers Packaging,"Klein Brothers Packaging, 1515 S Fresno Ave, S...",0x80901260b5987b39:0x8a5028c253e7bee6,None,37.934318,-121.313102,[Trucking company],4.2,3,None,"[[Saturday, 10AM–1PM], [Sunday, Closed], [Mond...",None,Open ⋅ Closes 1PM,"[0x80900c2bd9b41a37:0x5de883f3a0746a61, 0x8090...",https://www.google.com/maps/place//data=!4m2!3...
3025008,Profiles Hair Design,"Profiles Hair Design, 522 N Federal Blvd, Rive...",0x8758dd1e2533c991:0x5f6d112918f07dba,None,43.029664,-108.380849,"[Beauty salon, Hair salon]",4.7,76,None,"[[Saturday, 9AM–4PM], [Sunday, Closed], [Monda...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 4PM,"[0x8758dd1ffba87d8f:0xf211a1ebf7297000, 0x8758...",https://www.google.com/maps/place//data=!4m2!3...
3025009,Arizona LINE-X,"Arizona LINE-X, 21242 N Black Canyon Hwy #A, P...",0x872b6f6f24aea445:0x22df9e5dd01e67ba,None,33.680066,-112.113367,"[Truck accessories store, Powder coating service]",4.6,24,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...","{'Accessibility': None, 'Activities': None, 'A...",Closed ⋅ Opens 8AM Mon,"[0x872b6f9d1b41ad95:0x94a335fc923f0987, 0x872b...",https://www.google.com/maps/place//data=!4m2!3...


En este caso, debo buscar en cuales filas aparece el nombre de "Holiday Inn" en las columnas de name o addresss ya que estas ofrecen más información en cuanto al nombre de los comercios.

In [10]:
hotel = ['Holiday Inn','holiday inn','Holiday inn', 'HolidayInn']

df_holiday = df_met[df_met['name'].str.contains('|'.join(hotel)) | df_met['address'].str.contains('|'.join(hotel))]

print(df_holiday)

                                                name  \
10780                     Holiday Inn Club Vacations   
10990              Holiday Inn Fitchburg - Waterpark   
37353                     Holiday Inn Club Vacations   
37563              Holiday Inn Fitchburg - Waterpark   
71118        Holiday Inn Express & Suites Fultondale   
...                                              ...   
2911673  Holiday Inn Express Cincinnati-Lawrenceburg   
2923748        Holiday Inn Express & Suites Hannibal   
2941963                                  Holiday Inn   
2950848                   Holiday Inn Express Bryant   
2958324                   Holiday Inn Club Vacations   

                                                   address  \
10780    Holiday Inn Club Vacations, 3000 S Ocean Blvd,...   
10990    Holiday Inn Fitchburg - Waterpark, 150 Great W...   
37353    Holiday Inn Club Vacations, 3000 S Ocean Blvd,...   
37563    Holiday Inn Fitchburg - Waterpark, 150 Great W...   
71118    Holiday 

In [11]:
df_holiday

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
10780,Holiday Inn Club Vacations,"Holiday Inn Club Vacations, 3000 S Ocean Blvd,...",0x890069fb0516e5e3:0x385b386ce00f5769,None,33.661750,-78.919719,[Travel agency],4.8,8,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x890069c0f5321335:0xc9a4295e5990da3c, 0x8900...",https://www.google.com/maps/place//data=!4m2!3...
10990,Holiday Inn Fitchburg - Waterpark,"Holiday Inn Fitchburg - Waterpark, 150 Great W...",0x89e3e40a2a7d4311:0x56b54dc03b117440,None,42.547034,-71.854357,"[Hotel, Convention center, Water park]",2.5,2,None,None,None,None,None,https://www.google.com/maps/place//data=!4m2!3...
37353,Holiday Inn Club Vacations,"Holiday Inn Club Vacations, 3000 S Ocean Blvd,...",0x890069fb0516e5e3:0x385b386ce00f5769,None,33.661750,-78.919719,[Travel agency],4.8,8,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x890069c0f5321335:0xc9a4295e5990da3c, 0x8900...",https://www.google.com/maps/place//data=!4m2!3...
37563,Holiday Inn Fitchburg - Waterpark,"Holiday Inn Fitchburg - Waterpark, 150 Great W...",0x89e3e40a2a7d4311:0x56b54dc03b117440,None,42.547034,-71.854357,"[Hotel, Convention center, Water park]",2.5,2,None,None,None,None,None,https://www.google.com/maps/place//data=!4m2!3...
71118,Holiday Inn Express & Suites Fultondale,"Holiday Inn Express & Suites Fultondale, 1733 ...",0x88890491e9416625:0x2f85307a06ddb16,Unfussy rooms & suites in a relaxed hotel offe...,33.609814,-86.807733,[Hotel],3.7,38,None,None,None,None,"[0x88890492ef4ca701:0x34f0c46c887eaf73, 0x8889...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911673,Holiday Inn Express Cincinnati-Lawrenceburg,"Holiday Inn Express Cincinnati-Lawrenceburg, 7...",0x8841d6dace0db59f:0x6f68b4bae01a7820,Standard lodging with an indoor pool & fitness...,39.115934,-84.852463,[Hotel],3.4,58,None,None,None,None,"[0x8841d1494aa6cb83:0xa61c2b85b39d62e2, 0x8841...",https://www.google.com/maps/place//data=!4m2!3...
2923748,Holiday Inn Express & Suites Hannibal,"Holiday Inn Express & Suites Hannibal, 4000 Ma...",0x87dde8ff6f0e5381:0x1b6e437aa892f148,Contemporary lodging with an indoor pool & fit...,39.686046,-91.402130,[Hotel],3.9,65,None,None,None,None,"[0x87dde8ff0b66b097:0x71404ed8104f93a7, 0x87dd...",https://www.google.com/maps/place//data=!4m2!3...
2941963,Holiday Inn,"Holiday Inn, 1000 Roosevelt Ave, Carteret, NJ ...",0x89c3b3062e30612f:0x99f57d78be6df52d,None,40.592759,-74.230818,[Bar],5.0,1,None,None,"{'Accessibility': None, 'Activities': None, 'A...",None,None,https://www.google.com/maps/place//data=!4m2!3...
2950848,Holiday Inn Express Bryant,"Holiday Inn Express Bryant, 2915 Main St, Brya...",0x87cd57abe07b5657:0x507855089383ddb,"Modern base with free breakfast, snacks, coffe...",34.621479,-92.496749,[Hotel],3.3,8,None,None,None,None,"[0x87d28217b4c735e5:0xf1f5a0e80cd5fe27, 0x87d2...",https://www.google.com/maps/place//data=!4m2!3...


Quiero extraer los gmap_id de este df resultante, para hacer la prueba de hallar las reviews correspondientes segun la coincidencia del id en ambos archivos.
Por lo cual, una vez enlistados los gmap_id procedo a recorrer la muestra del dataframe y corroborar la informacion.

In [12]:
lista_id_metadata = df_holiday['gmap_id'].tolist()
lista_id_metadata

['0x890069fb0516e5e3:0x385b386ce00f5769',
 '0x89e3e40a2a7d4311:0x56b54dc03b117440',
 '0x890069fb0516e5e3:0x385b386ce00f5769',
 '0x89e3e40a2a7d4311:0x56b54dc03b117440',
 '0x88890491e9416625:0x2f85307a06ddb16',
 '0x89c3c096710cef79:0x198c9a49b43f1e7e',
 '0x80c29d92d2ee7be3:0x83470e9ebf3df73e',
 '0x87b6f18f7db7a55f:0x32b127e319e0e54',
 '0x8640bf3e578cc269:0xcb90dd8496e825a1',
 '0x89b7629b154cfbab:0x3cf0fde8b6c5ab3e',
 '0x881c1a05c55bfbf7:0xa93a510f8bd050df',
 '0x87c01b06036d90a5:0xa52fae6d5b389fc5',
 '0x88404efaf0af8277:0xf2421ac24353e222',
 '0x8840519f819c4ac1:0x716391e2e7e4cdba',
 '0x8796b94989928343:0x77b98bf58f3f4e49',
 '0x87cf1be642c481d7:0x7c3adb844552d74e',
 '0x889386ff5c9a904d:0x8bb251e14c3e80f4',
 '0x883333581a9e9547:0x53724d0cb64b5a62',
 '0x88d9c7fe5165cfab:0x48df90f0bd29376b',
 '0x89b637ff613de4cd:0x8174b26d6c30fb36',
 '0x8776c2790bfa7ffd:0x7b3bc3df38f06534',
 '0x89c698a0875f83bf:0x6ecf8a65960b28',
 '0x87f7d60e2a419d6f:0x3dd046f00177c38b',
 '0x8651b140bbf7cb33:0xb81531a2c8f17f6

In [13]:
#filtro el df de muestra, quedandome con las filas donde coincida cualquiera de los gmap_id de metadata
filter_review = df[df['gmap_id'].isin(lista_id_metadata)]
print(filter_review)

                      user_id               name  \
4637    112665186528292597385      Charlie Cline   
4638    108637957038201408174  Jennifer Passmore   
4639    109079098695645726629     TheBeckyboo926   
4640    117088716850554427602   yolanda arellano   
4641    117028379331677388352       Greg Randall   
...                       ...                ...   
731631  101429772691670071930        Blair Waite   
731632  116882710014781503283       James Harris   
731633  114651913262418355510    Jonathan Hadway   
731634  111786927781973217171     Alonzo Coleman   
731635  110973657083808837962     Rebecca Morgan   

                                   time  rating  \
4637   2016-02-07 04:58:14.934000+00:00       4   
4638   2017-04-12 03:04:16.419000+00:00       5   
4639   2016-10-15 01:14:34.021000+00:00       5   
4640   2016-11-14 14:12:46.153000+00:00       3   
4641   2016-05-28 18:39:33.963000+00:00       4   
...                                 ...     ...   
731631 2017-07-17 

Visualizo la información

In [14]:
filter_review

,user_id,name,time,rating,text,pics,resp,gmap_id
4637,112665186528292597385,Charlie Cline,2016-02-07 04:58:14.934000+00:00,4,Good location. Nice staff. Clean. Breakfast ar...,None,None,0x88890491e9416625:0x2f85307a06ddb16
4638,108637957038201408174,Jennifer Passmore,2017-04-12 03:04:16.419000+00:00,5,Super clean very friendly,None,None,0x88890491e9416625:0x2f85307a06ddb16
4639,109079098695645726629,TheBeckyboo926,2016-10-15 01:14:34.021000+00:00,5,Amazing! We had a conference room and it was a...,None,None,0x88890491e9416625:0x2f85307a06ddb16
4640,117088716850554427602,yolanda arellano,2016-11-14 14:12:46.153000+00:00,3,A bit on the pricey side but they take on pets...,None,None,0x88890491e9416625:0x2f85307a06ddb16
4641,117028379331677388352,Greg Randall,2016-05-28 18:39:33.963000+00:00,4,Solid hotel. Staff was friendly. Room was clean.,None,None,0x88890491e9416625:0x2f85307a06ddb16
...,...,...,...,...,...,...,...,...
731631,101429772691670071930,Blair Waite,2017-07-17 17:42:34.935000+00:00,2,None,None,None,0x88891ed75ed74a45:0xf7f56f0dc77c3f96
731632,116882710014781503283,James Harris,2019-01-11 17:50:26.259000+00:00,5,None,None,None,0x88891ed75ed74a45:0xf7f56f0dc77c3f96
731633,114651913262418355510,Jonathan Hadway,2019-01-15 16:31:08.840000+00:00,5,None,None,None,0x88891ed75ed74a45:0xf7f56f0dc77c3f96
731634,111786927781973217171,Alonzo Coleman,2018-10-18 03:54:08.402000+00:00,3,None,None,None,0x88891ed75ed74a45:0xf7f56f0dc77c3f96


Una vez chequeada la funcionalidad de cada paso por separado, creo una función cual va a unificar todos estos procesos y va a recorrer carpeta por carpeta para separarme con las reviews de Holiday Inn.

Defino también la función que me permite guardar estos datos directamente en un .parquet, cada archivo con el nombre de su estado correspondiente.

In [15]:
def conversor_parquet(dataframe, name_folder):
    name_parquet = f"{name_folder}.parquet"
    dataframe.to_parquet(name_parquet, index=False)
    print(f"DataFrame guardado en '{name_parquet}'")

En la siguiente funcion voy a llamar a todas las funciones anteriores para unificar el trabajo de selección.

In [16]:
def iterador_carpeta(carpeta_raiz, df):
    lista_id_metadata = df['gmap_id'].tolist()
    for ruta_actual, carpetas_sub, _ in os.walk(carpeta_raiz):
        for carpeta_sub in carpetas_sub:
            ruta_carpeta_sub = os.path.join(ruta_actual, carpeta_sub)
            dataframe = iterador_archivo(ruta_carpeta_sub)
            dataframe_filtrado = filter_year(dataframe)
            filter_review = dataframe_filtrado[dataframe_filtrado['gmap_id'].isin(lista_id_metadata)]
            
            conversor_parquet(filter_review, ruta_carpeta_sub)

In [17]:
carpeta_principal = "../reviews-estados"

In [19]:
iterador_carpeta(carpeta_principal, df_holiday)

DataFrame guardado en '../reviews-estados\review-Alabama.parquet'
DataFrame guardado en '../reviews-estados\review-Alaska.parquet'
DataFrame guardado en '../reviews-estados\review-Arizona.parquet'
DataFrame guardado en '../reviews-estados\review-Arkansas.parquet'
DataFrame guardado en '../reviews-estados\review-California.parquet'
DataFrame guardado en '../reviews-estados\review-Colorado.parquet'
DataFrame guardado en '../reviews-estados\review-Connecticut.parquet'
DataFrame guardado en '../reviews-estados\review-Delaware.parquet'
DataFrame guardado en '../reviews-estados\review-District_of_Columbia.parquet'
DataFrame guardado en '../reviews-estados\review-Florida.parquet'
DataFrame guardado en '../reviews-estados\review-Georgia.parquet'
DataFrame guardado en '../reviews-estados\review-Hawaii.parquet'
DataFrame guardado en '../reviews-estados\review-Idaho.parquet'
DataFrame guardado en '../reviews-estados\review-Illinois.parquet'
DataFrame guardado en '../reviews-estados\review-Indiana